In [60]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np


In [77]:
#from plot_lib import plot_data, plot_model, set_default

In [76]:
#set_default()

In [80]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np

# function to count number of parameters
def get_n_params(model):
    np=0
    for p in list(model.parameters()):
        np += p.nelement()
    return np

In [81]:
input_size  = 28*28   # 28x28 pixels
output_size = 10      # 10 classes

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=64, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=1000, shuffle=True)

# 4.1.New Model Class

In [82]:
class CNN(nn.Module):
    def __init__(self,conv_kernel_size, pooling_kernel_size, stride_size, zero_padding, max_pooling):
        super(CNN, self).__init__()
        self.conv_kernel_size = conv_kernel_size
        self.pooling_kernel_size = pooling_kernel_size
        self.stride_size = stride_size
        self.zero_padding = zero_padding
        self.max_pooling = max_pooling
        
        if zero_padding is True:
            padding_size = 2
            d = 7
        else:
            padding_size = 0
            d = 4
        self.conv1 = nn.Conv2d(1, 16, kernel_size=conv_kernel_size,stride=stride_size, padding=padding_size)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=conv_kernel_size,stride=stride_size, padding=padding_size)
        self.fc1 = nn.Linear(32 * d * d, 10)
     
    def forward(self, x, verbose=False):
        if self.max_pooling is True:
            x = self.conv1(x)
            x = F.max_pool2d(x, kernel_size=self.pooling_kernel_size)
            x = self.conv2(x)
            x = F.max_pool2d(x, kernel_size=self.pooling_kernel_size)
            x = x.view(-1, 32*4*4)
            x = self.fc1(x)
            x = F.log_softmax(x, dim=1)
        else:
            x = self.conv1(x)
            x = F.avg_pool2d(x, kernel_size=self.pooling_kernel_size)
            x = self.conv2(x)
            x = F.avg_pool2d(x, kernel_size=self.pooling_kernel_size)
            x = x.view(-1, 32*4*4)
            x = self.fc1(x)
            x = F.log_softmax(x, dim=1)
            
        return x

# 4.2. Zero Padding Model Tensors

In [87]:
cnn = CNN(5,2,1, True, True)
print(cnn)

CNN(
  (conv1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (fc1): Linear(in_features=1568, out_features=10, bias=True)
)


In [88]:
params = np.array(list(cnn.parameters()))
print(params[0].size()) # conv 1
print(params[1].size()) # conv 1 bias
print(params[2].size()) # conv 2
print(params[3].size()) # conv 2 bias
print(params[4].size()) # fc 
print(params[5].size()) # fc bias

torch.Size([16, 1, 5, 5])
torch.Size([16])
torch.Size([32, 16, 5, 5])
torch.Size([32])
torch.Size([10, 1568])
torch.Size([10])


# 4.3.Valid Padding Model Tensors

In [74]:
cnn = CNN(5,2,1, False, True)

In [91]:
print(cnn)

CNN(
  (conv1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (fc1): Linear(in_features=1568, out_features=10, bias=True)
)


In [92]:
params = list(cnn.parameters())
print(params[0].size()) # conv 1
print(params[1].size()) # conv 1 bias
print(params[2].size()) # conv 2
print(params[3].size()) # conv 2 bias
print(params[4].size()) # fc 
print(params[5].size()) # fc bias

torch.Size([16, 1, 5, 5])
torch.Size([16])
torch.Size([32, 16, 5, 5])
torch.Size([32])
torch.Size([10, 1568])
torch.Size([10])
